Data Wrangling Notebook for Famoso et al 2013 data
<br />
Neeka Sewnath
<br />
nsewnath@ufl.edu

In [4]:
import pandas as pd
import numpy as np
import uuid

Silence unnecessary errors

In [5]:
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    pass

Notes

In [6]:
#What is sheet 2 in the spreadsheet?

Import Famoso et al 2013 data

In [7]:
data = pd.read_csv("../Original_Data/Famoso_et_al_2013.csv")

Create verbatimScientificName

In [ ]:
data = data.assign(verbatimScientificName = data["Genus"] + " " + data["Species"])
data = data.assign(scientificName = data["verbatimScientificName"])

Adding additional required GEOME columns

In [ ]:
#TODO: Check these again
data = data.assign(country = "Unknown", yearCollected = "Unknown")

Set samplingProtocol and measurementMethod 

In [ ]:
citation = "Famoso, N. A., Feranec, R. S., & Davis, E. B. (2013). Occlusal enamel complexity and its implications for lophodonty, hypsodony, body mass, and diet in extinct and extant ungulates. Palaeogeography, Palaeoclimatology, Palaeoecology, 387, 211–216. https://doi.org/10.1016/j.palaeo.2013.07.006"

data = data.assign(samplingProtocol = citation, measurementMethod = citation)

Rearrange columns so that template columns are first, followed by measurement values

In [ ]:
# Create column list
cols = data.columns.tolist()

# Specify desired columns
cols = ['country',
        'yearCollected',
        'samplingProtocol',
        'measurementMethod',
        'verbatimScientificName',
        'scientificName']

# Subset dataframe
data = data[cols]

Create necessary materialSampleID column and populate with UUID (use hex to remove dashes). 

In [ ]:
data = data.assign(materialSampleID = '')
data['materialSampleID'] = [uuid.uuid4().hex for _ in range(len(data.index))]

Create a long version of the data frame

In [ ]:
# Creating long version, first specifiying keep variables, then naming variable and value
longVers = pd.melt(data, id_vars = ['country',
                                    'yearCollected',
                                    'samplingProtocol',
                                    'measurementMethod',
                                    'verbatimScientificName',
                                    'scientificName'], 
                                    var_name = 'measurementType', 
                                    value_name = 'measurementValue')

Create diagnosticID which is a unique number for each measurement

In [ ]:
longVers = longVers.assign(diagnosticID = '')
longVers['diagnosticID'] = np.arange(len(longVers))

If measurement value equals N/a, delete entire row

In [ ]:
longVers = longVers.dropna(subset = ['measurementValue'])

Writing long data csv file

In [ ]:
longVers.to_csv('../Mapped_Data/Famoso_et_al_2013.csv', index = False)